In [1]:
import tensorflow as tf
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
def load_imdb():
  # download dataset
  url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

  dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

  dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
  train_dir = os.path.join(dataset_dir, 'train')
  test_dir = os.path.join(dataset_dir, 'test')
  
  # remove irrelevant data
  remove_dir = os.path.join(train_dir, 'unsup')
  shutil.rmtree(remove_dir)

  # load to dataframes
  train_lst, test_lst = [], []
  label2id = {"pos" : 1, "neg" : 0}

  for label in ['pos', 'neg']:
    path = train_dir + "/" + label
    files = os.listdir(path)
    for _file in files:
      with open(os.path.join(path, _file), 'r') as f:
        # strip <br /> tags
        text = f.read()
        train_lst.append([text, label2id[label]])
    
    path = test_dir + "/" + label
    files = os.listdir(path)
    for _file in files:
      with open(os.path.join(path, _file), 'r') as f:
        text = f.read()
        test_lst.append([text, label2id[label]])
    
  df_train = pd.DataFrame(train_lst, columns=['text', 'label'])
  df_test  = pd.DataFrame(test_lst, columns=['text', 'label'])
  x_train, y_train = df_train["text"], df_train["label"]
  x_test, y_test = df_test["text"], df_test["label"]

  return x_train, y_train, x_test, y_test

In [3]:
def load_fin():
  # download dataset
  url = '/kaggle/input/financial-sentiment-analysis/data.csv'

  # load to dataframes
  df_raw = pd.read_csv(url)
  label2id = {"positive" : 2, "neutral" : 1, "negative" : 0}
  df_raw["Sentiment"] = df_raw["Sentiment"].apply(lambda x : label2id[x])
    
  df_train, df_test = train_test_split(df_raw)
  x_train, y_train = df_train["Sentence"], df_train["Sentiment"]
  x_test, y_test = df_test["Sentence"], df_test["Sentiment"]

  return x_train, y_train, x_test, y_test

In [4]:
def load_sst5():
    train_url = 'https://raw.githubusercontent.com/christycty/sentiment-analysis-review/main/data/sst5_train.csv'
    test_url = 'https://raw.githubusercontent.com/christycty/sentiment-analysis-review/main/data/sst5_test.csv'
    
    df_train = pd.read_csv(train_url)
    df_test = pd.read_csv(test_url)
    
    x_train, y_train = df_train["sentence"], df_train["label"]
    x_test, y_test = df_test["sentence"], df_test["label"]

    return x_train, y_train, x_test, y_test

In [5]:
def load_sst2():
    train_url = 'https://raw.githubusercontent.com/christycty/sentiment-analysis-review/main/data/sst5_train.csv'
    test_url = 'https://raw.githubusercontent.com/christycty/sentiment-analysis-review/main/data/sst5_test.csv'
    
    df_train = pd.read_csv(train_url)
    df_test = pd.read_csv(test_url)
    
    # remove neutral
    df_train = df_train[df_train["label"] != 2]
    df_test = df_test[df_test["label"] != 2]
    
    # map to positive or negative
    label2id = {0:0, 1:0, 3:1, 4:1}
    df_train["label"] = df_train["label"].apply(lambda x : label2id[x])
    df_test["label"] = df_test["label"].apply(lambda x : label2id[x])
    
    x_train, y_train = df_train["sentence"], df_train["label"]
    x_test, y_test = df_test["sentence"], df_test["label"]

    return x_train, y_train, x_test, y_test

In [6]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import tensorflow_hub as hub

def get_embeddings(x_train, output_len=128):
    embed = TextVectorization(max_tokens=10000, output_mode='int', 
                                        output_sequence_length=output_len)
    embed.adapt(x_train)
    return embed

In [7]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re

def preprocess(text_inp):
    TAG_RE = re.compile(r'<[^>]+>')

    text = TAG_RE.sub('', text_inp)
    text = re.sub('[^a-zA-Z]', ' ', text) # non alphabets
    text = re.sub(r'\s+', ' ', text)  # multiple space
    
    # stopwords
    text = text.lower().split()
    stopwords_set = set(stopwords.words('english'))
    text = [x for x in text if x not in stopwords_set]
    return " ".join(text)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [15]:
def build_cnn(num_class, in_len=128):
    model = Sequential([
        tf.keras.layers.Embedding(10000, 16, input_length=in_len),
        tf.keras.layers.Conv1D(filters = 32, kernel_size = 7, activation = "relu"),
        tf.keras.layers.MaxPooling1D(5),
        tf.keras.layers.Conv1D(filters = 32, kernel_size = 7, activation = "relu"),
        tf.keras.layers.MaxPooling1D(3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.5),
    ])
    
    if (num_class == 2):
        net = tf.keras.layers.Dense(1, activation='sigmoid')
    else:
        net = tf.keras.layers.Dense(num_class, activation='softmax')
    model.add(net)
    
    if num_class == 2:
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    else:
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [10]:
def build_rnn(num_class, in_len=128):
    model = Sequential([
        tf.keras.layers.Embedding(10000, 16, input_length=in_len),
        tf.keras.layers.SimpleRNN(64, dropout=0.5),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.5),
    ])
    if (num_class == 2):
        net = tf.keras.layers.Dense(1, activation='sigmoid')
    else:
        net = tf.keras.layers.Dense(num_class, activation='softmax')
    model.add(net)
    
    if num_class == 2:
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    else:
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [11]:
def build_lstm(num_class, in_len=128):
    model = Sequential([
        tf.keras.layers.Embedding(10000, 16, input_length=in_len),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, dropout=0.5)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.5),
    ])
    
    if (num_class == 2):
        net = tf.keras.layers.Dense(1, activation='sigmoid')
    else:
        net = tf.keras.layers.Dense(num_class, activation='softmax')
    model.add(net)
    
    if num_class == 2:
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    else:
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
  

In [12]:
def train(model, x_train, y_train, x_val, y_val, epochs=10):
  # Train the model
  history = model.fit(x_train, y_train, epochs=epochs,
                      validation_data=(x_val, y_val))

  # Evaluate the model on the validation set
  loss, accuracy = model.evaluate(x_val, y_val)
  print(f'Validation loss: {loss:.4f}, Validation accuracy: {accuracy:.4f}')
  return history

In [1]:
def test_model(data, model_name):
    if data == "fin":
        x_train_raw, y_train_, x_test_raw, y_test = load_fin()
        num_classes = 3
    elif data == "imdb":
        x_train_raw, y_train_, x_test_raw, y_test = load_imdb()
        num_classes = 2
    elif data == "sst5":
        x_train_raw, y_train_, x_test_raw, y_test = load_sst5()
        num_classes = 5
    elif data == "sst2":
        x_train_raw, y_train_, x_test_raw, y_test = load_sst2()
        num_classes = 2
    
    x_train_ = x_train_raw.apply(preprocess)
    x_test_ = x_test_raw.apply(preprocess)
    
    maxlen = 500
    embed = get_embeddings(x_train_)
    x_train_ = embed(x_train_)
    x_test = embed(x_test_)
    
    x_train, x_val, y_train, y_val = train_test_split(x_train_.numpy(), y_train_, test_size=0.2)
    
    print(y_train[:10])
    print(x_train.shape, x_val.shape)
    
    if model_name == "rnn":
        model = build_rnn(num_classes)
    elif model_name == "lstm":
        model = build_lstm(num_classes)
    elif model_name == "cnn":
        model = build_cnn(num_classes)
        
    print(model)
        
    hist = train(model, x_train, y_train, x_val, y_val)
    
    model.evaluate(x_test, y_test)
    model_save = f"/kaggle/working/{data}_{model_name}"
    
#     model.save(model_save)
#     shutil.make_archive(model_save, 'zip', "/kaggle/working")
    
    hist_df = pd.DataFrame(hist.history)
    hist_df.to_csv(model_save + "_hist.csv")

In [17]:
test_model('imdb', 'cnn')

19970    0
12531    0
23754    0
14206    0
17305    0
11094    1
15485    0
17552    0
15691    0
7438     1
Name: label, dtype: int64
(20000, 128) (5000, 128)
Epoch 1/10
625/625 [==============================] - 60s 84ms/step - loss: 0.5419 - accuracy: 0.7007 - val_loss: 0.4624 - val_accuracy: 0.7632
Epoch 2/10
625/625 [==============================] - 14s 22ms/step - loss: 0.4100 - accuracy: 0.8022 - val_loss: 0.4673 - val_accuracy: 0.7639
Epoch 3/10
625/625 [==============================] - 9s 14ms/step - loss: 0.3590 - accuracy: 0.8276 - val_loss: 0.4822 - val_accuracy: 0.7618
Epoch 4/10
625/625 [==============================] - 6s 9ms/step - loss: 0.3013 - accuracy: 0.8546 - val_loss: 0.5193 - val_accuracy: 0.7535
Epoch 5/10
625/625 [==============================] - 5s 8ms/step - loss: 0.2357 - accuracy: 0.8842 - val_loss: 0.6595 - val_accuracy: 0.7461
Epoch 6/10
625/625 [==============================] - 6s 9ms/step - loss: 0.1739 - accuracy: 0.9074 - val_loss: 0.8281 - val

In [20]:
test_model('sst2', 'cnn')

3831    1
7819    0
604     1
8542    0
8313    1
2079    1
1743    1
1771    1
5177    0
1263    1
Name: label, dtype: int64
(5536, 128) (1384, 128)
Epoch 1/10
173/173 [==============================] - 16s 82ms/step - loss: 0.6924 - accuracy: 0.5227 - val_loss: 0.6925 - val_accuracy: 0.5101
Epoch 2/10
173/173 [==============================] - 4s 21ms/step - loss: 0.6794 - accuracy: 0.5347 - val_loss: 0.6670 - val_accuracy: 0.5499
Epoch 3/10
173/173 [==============================] - 3s 16ms/step - loss: 0.6236 - accuracy: 0.5861 - val_loss: 0.6735 - val_accuracy: 0.5388
Epoch 4/10
173/173 [==============================] - 2s 12ms/step - loss: 0.5950 - accuracy: 0.5966 - val_loss: 0.6996 - val_accuracy: 0.5533
Epoch 5/10
173/173 [==============================] - 2s 10ms/step - loss: 0.5807 - accuracy: 0.6064 - val_loss: 0.7477 - val_accuracy: 0.5517
Epoch 6/10
173/173 [==============================] - 2s 10ms/step - loss: 0.5746 - accuracy: 0.6081 - val_loss: 0.7999 - val_accuracy

In [24]:
test_model('fin', 'cnn')

3039    2
629     1
3232    2
334     1
4139    0
1421    1
24      1
1911    0
2193    1
4391    0
Name: Sentiment, dtype: int64
(3504, 128) (877, 128)
Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert' defined at (most recent call last):
    File "/opt/conda/lib/python3.7/runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "/opt/conda/lib/python3.7/runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.7/site-packages/traitlets/config/application.py", line 1041, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.7/asyncio/base_events.py", line 541, in run_forever

In [25]:
test_model('sst5', 'cnn')

170     2
4768    2
3049    2
736     4
91      3
1722    3
4733    2
5489    1
955     4
7658    0
Name: label, dtype: int64
(6835, 128) (1709, 128)
Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert' defined at (most recent call last):
    File "/opt/conda/lib/python3.7/runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "/opt/conda/lib/python3.7/runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.7/site-packages/traitlets/config/application.py", line 1041, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.7/asyncio/base_events.py", line 541, in run_forever